In [23]:
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (load_wine())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])

In [25]:
print(x.shape,y.shape) # (178, 13) (178, 1)
print(y[:5])

# y의 클래스 값 분류 + 불균형확인
print(np.unique(y)) # [0 1 2]
print(np.unique(y,return_counts=True)) # [0 1 2]

y = to_categorical(y)

(178, 13) (178, 1)
   0
0  0
1  0
2  0
3  0
4  0
[0 1 2]
(array([0, 1, 2]), array([59, 71, 48], dtype=int64))


In [26]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )
# print(x.columns)
# y 의 첫번째 칼럼을 원핫 인코딩 변환
# y_train[0] = tf.one_hot(y_train[0],3)
# y_test[0] = tf.one_hot(y_test[0],3)
print(y_train[:5])
print("shape : ",y_train.shape)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
shape :  (142, 3)


In [80]:
model = Sequential([
    Dense(50,activation="relu",input_shape=(13,)),
    Dense(40,activation="relu") ,
    Dense(40,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(20,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [81]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=5, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
3/3 [==============================] - 0s 69ms/step - loss: 1.1164 - accuracy: 0.3438 - val_loss: 1.1085 - val_accuracy: 0.2917
Epoch 2/100
3/3 [==============================] - 0s 14ms/step - loss: 1.0821 - accuracy: 0.5208 - val_loss: 1.0764 - val_accuracy: 0.6250
Epoch 3/100
3/3 [==============================] - 0s 14ms/step - loss: 1.0573 - accuracy: 0.6146 - val_loss: 1.0543 - val_accuracy: 0.6250
Epoch 4/100
3/3 [==============================] - 0s 13ms/step - loss: 1.0336 - accuracy: 0.6042 - val_loss: 1.0299 - val_accuracy: 0.6250
Epoch 5/100
3/3 [==============================] - 0s 14ms/step - loss: 1.0095 - accuracy: 0.6562 - val_loss: 0.9967 - val_accuracy: 0.6250
Epoch 6/100
3/3 [==============================] - 0s 14ms/step - loss: 0.9651 - accuracy: 0.6771 - val_loss: 0.9510 - val_accuracy: 0.6250
Epoch 7/100
3/3 [==============================] - 0s 14ms/step - loss: 0.9074 - accuracy: 0.6771 - val_loss: 0.8926 - val_accuracy: 0.6250
Epoch 8/100
3/3 [===

In [82]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


1/1 [==============================] - 0s 17ms/step - loss: 0.0903 - accuracy: 0.9667
loss: 0.09032515436410904 
acc : 0.9666666388511658


In [83]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
# y_predict = to_categorical(y_predict,3)
print(y_predict[:5])
# y_test 를 원핫 인코딩 해제 해야함
y_test = np.argmax(y_test,axis=1)
print(y_test[:5])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)

[[9.9883050e-01 1.1695052e-03 1.9535790e-08]
 [9.9652570e-01 3.4741717e-03 1.3695870e-07]
 [5.7510961e-06 1.1239207e-02 9.8875511e-01]
 [9.9739194e-01 2.6079724e-03 9.2927820e-08]
 [4.6181932e-04 7.7859813e-01 2.2094001e-01]]
[0 0 2 0 1]
[0 0 2 0 1]
0.9666666666666667
